# Kindle Note Caster


In [1]:
### import

import pandas as pd
import numpy as np
import docx
import json
import re

In [2]:
### Nom des chemins:

i_chemin_du_csv= "C.csv"
o_chemin_du_word= "The_Customer_Success_Professionals_Handbook.docx"


In [3]:
### fonctions:

def ajouter_surlignement(surlignement):
    global o_texte
    if (str(surlignement) not in List_mot_a_enlever):
        o_texte = o_texte +'\n\n'+ str(surlignement)

def ajouter_note(note):
    global o_texte
    if (str(note) not in List_mot_a_enlever):
        o_texte = o_texte +' => '+ str(note)
        
def num_emplacement(emplacement):
    """not used"""
    return int(emplacement[12:])

def check_hashtag(note):
    if '@@' in note:
        return True
    return False

def extract_keyword(text):
    # example: sales de "##sales commentaire1 commentaire2"
    pattern=re.compile(r"""(?<=@@)([A-ù]+)(?= )""")
    match = re.search(pattern, text)
    if match:
        return(match[0].lower())
    print("error: keyword  not found")
    
def extract_note(text):
    # example: "commentaire1 commentaire2" de "##sales commentaire1 commentaire2"
        return(text.replace('@@'+extract_keyword(text),'=>'))
    
def ajouter_au_fichier(filepath, surlignement, note ):
    #ajout de la note et du surlignement
    #preprocessing du texte
    output_texte = surlignement + extract_note(note)
    #ajout à la fin du texte
    mydoc = docx.Document(filepath)
    mydoc.paragraphs[-1]
    mydoc.add_paragraph(output_texte)
    mydoc.save(filepath)


In [4]:
### script
df=pd.read_csv(i_chemin_du_csv, sep=',') #on_bad_lines{‘error’, ‘warn’, ‘skip’}
df

,Vos notes Kindle pour�:,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,THE CUSTOMER SUCCESS PROFESSIONAL'S HANDBOOK: ...,NaN,NaN,NaN
1,"de Ashvin Vaidyanathan, Ruben Rabago",NaN,NaN,NaN
2,Aper�u instantan� Kindle gratuit�:,NaN,NaN,NaN
3,https://amzn.eu/9RsToTG,NaN,NaN,NaN
4,----------------------------------------------,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,Type d'annotation,Emplacement,Suivi�?,Annotation
7,Surlignement (Jaune),Emplacement 575,NaN,acquiring new logos.
8,Note,Emplacement 576,NaN,@@sales Expression
9,Surlignement (Jaune),Emplacement 590,NaN,"Customer Success is a new mindset, and the rol..."


In [5]:
o_texte = ""
List_mot_a_enlever=['nan','Annotation'] # ce son en fait les titres des colonnes (cf amélioration possible)

In [6]:
#load file_mapper
file_mapper = json.load(open('file_mapper.json'))
#rename columns "Type d'annotation","Emplacement","Suivi ?","Annotation"
df.columns = ['Type','Emplacement','unknown','texte']

In [7]:
for i in df.index: 
    
    if df['Type'][i]=='Surlignement (Jaune)':
        ajouter_surlignement(df['texte'][i])
        
    elif df['Type'][i]=='Note':
        #vérifie qu'il n'y a pas de ##
        
        if not check_hashtag(df['texte'][i]):
            ajouter_note(df['texte'][i])
                
        else:
            # il y a bien un marquage de rediretion de notes: il faut placer la note dans le bon fichier du dossier Lecture
            # ce mapping (le bon endroit des fichiers) est stocké dans un dictionnaire
            # celui ci est documenté dans le README
            # ajouter dans le fichier concerné la note + l'emplacement précédent

            if extract_keyword(df['texte'][i]) in file_mapper.keys():
                ajouter_au_fichier(file_mapper[extract_keyword(df['texte'][i])] , df['texte'][i-1], df['texte'][i] )

# pas de else: les autres cas sont les résidus de texte en haut

In [8]:
mydoc = docx.Document()
mydoc.add_paragraph(o_texte)
mydoc.save(o_chemin_du_word)
